# Part 2, Zero Shot Sentiments

In [2]:
# !pip install ktrain

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 25.3 MB 39 kB/s  eta 0:00:014.6 MB/s eta 0:00:05          | 11.5 MB 4.6 MB/s eta 0:00:04        | 18.4 MB 4.6 MB/s eta 0:00:02��███████████████▊| 25.1 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 92.9 MB/s eta 0:00:01
     |████████████████████████████████| 19.2 MB 97.1 MB/s eta 0:00:01             | 5.3 MB 97.1 MB/s eta 0:00:01��████████████████████▋          | 12.9 MB 97.1 MB/s eta 0:00:01
     |████████████████████████████████| 265 kB 120.6 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 115.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 120.0 MB/s eta 0:00:01  |███████████▉                    | 1.0 MB 120.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 102.0 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 106.4 MB/s eta 0:00:01
  Created wheel for ktrain: filename=ktrain-0

  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3958 sha256=367f55a7360cf5dd3f8dbc3c64bc8f73667034b83e81cab0a10e2bf58ace4b1e
  Stored in directory: /home/vritti/.cache/pip/wheels/cb/25/02/4bb438785ef9c10d07f6b3519f080b38917153fdac3108d738
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=97f279ae1805c539ad7282c901fd8da83d31d1c98d8b65e32428235bcc3226df
  Stored in directory: /home/vritti/.cache/pip/wheels/c1/df/15/a88cdf68ce687574649f65063a743123e1bee79932b6eea3b6
  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14993 sha256=ea4313fc1796c2a15df9285dc6ab7fc377685fa37658ef8a65632d857fb7458f
  Stored in directory: /home/vritti/.cache/pip/wheels/b3/85/50/f232cac81ed1eb4dc20db31a9d1f4a8a1a8c696d4d27bff442
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.13.0-py3-none-any.whl size=6961 sha256=0ac7a7e90cef91d7168a046f7dd847

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
import ktrain

In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Initializing the zero shot classifier

zsl = ktrain.text.ZeroShotClassifier()

In [7]:
df_cleaned_news = pd.read_csv('df_cleaned_news.csv')
df_cleaned_news.head()

,Unnamed: 0,date,clean_text,clean_title
0,0,2022-01-06,chicago ap chicago school leaders canceled classes thursday second consecutive day failing reach agreement teachers union covid safety protocols nation third largest school district the chicago teachers union sought revert remote instruction latest surge infections sides hammer deal but chicago public schools leaders said remote learning work schools safely remain open protocols place the move cancel classes activities thursday affects roughly students ...,no deal school chicago cancels classes 2nd day
1,1,2022-01-06,chicago pd lies season episode promo voight employs new informant help solve tricky drug trafficking case atwater struggles reconcile personal professional life reaches decision jesseleesoffer marinasqu trspiridakos nbcchicagopd,watch chicago p d preview wednesday
2,2,2022-01-06,wise foolish the chicago tribune article titled chicago public schools cancels classes thursday deadlock union continues trump biden press secretary comment impasse reported trumps solution follows educate children person give every dollar spent education directly students get failing government schools trump said statement the purpose this post is relate ancient wisdom proverb trump solution king solomon go presence foolish man thou perceivest lips...,trump solution chicago public schools king solomon blog
3,3,2022-01-06,the neighbors near mashawn plummer portage park firehouse knew often sat outside friendly presence station he showed early shifts ready job thrilled friends family recalled chicago firefighter emt mashawn plummer engine chicago fire department hundreds chicago firefighters joined family friends thursday lay plummer rest died december battling apartment fire northwest side they remembered plummer gentle giant followed dream becoming chicago fi...,family friends colleagues gathering today south side church attend funeral services chicago firefighter mashawn plummer
4,4,2022-01-06,mashawn plummer died final month preventing fireplace town northwest aspect additionally killed civilian a visitation held plummer wednesday night time a remaining visitation probably held home hope church e 114th st related chicago firefighter amongst critically injured belmont central residence fireplace killed following ultimate visitation funeral companies start probably adopted procession oak woods cemetery 67th road the yea...,chicago firefighter funeral pals household say goodbye mashawn plummer


In [8]:
df_cleaned_news.isna().sum()

Unnamed: 0     0
date           0
clean_text     7
clean_title    6
dtype: int64

In [9]:
df_cleaned_news.dropna(inplace=True)

In [10]:
df_cleaned_news.shape

(165360, 4)

In [11]:
df_cleaned_news.reset_index(inplace=True, drop=True)

In [12]:
df_cleaned_news.shape

(165360, 4)

In [13]:
texts = df_cleaned_news['clean_text'].tolist()

In [14]:
texts[0]

'chicago  ap   chicago school leaders canceled classes thursday   second consecutive day  failing  reach agreement   teachers union  covid   safety protocols   nation third largest school district  the chicago teachers union sought  revert  remote instruction   latest surge  infections    sides hammer   deal  but chicago public schools leaders  said remote learning  work  schools  safely remain open  protocols  place  the move  cancel classes  activities thursday affects roughly     students  came  closed door negotiations wednesday failed  produce  deal  the issues include metrics  closing schools  we   choice   cancel classes tomorrow  schools ceo pedro martinez said   wednesday evening news conference  students returned  class monday   two week winter break  covid   infections  hospitalizations fueled   omicron variant  record levels  school districts nationwide  grappled    issue     opting  stay open  ramping  virus testing  tweaking protocols   adjustments  response   shifting pa

In [15]:
# Sample prediction with ZSL

labels=['positive', 'negative']
zsl.predict(texts[0], labels=labels, include_labels=False, nli_template="The sentiment of this article is {}.", multilabel=False)

[0.5120956301689148, 0.4879043400287628]

In [16]:
# Prediction positive/negative sentiments with ZSL

predictions = [zsl.predict(text, labels=labels, include_labels=False, \
                           nli_template="The sentiment of this news article is {}.") for text in texts]

In [17]:
predictions_df = pd.DataFrame(predictions, columns=[labels]) 

In [18]:
predictions_df.head()

,positive,negative
0,0.295749,0.957052
1,0.002615,0.986636
2,0.040025,0.966754
3,0.892183,0.033660
4,0.915046,0.068743


In [19]:
news_sentiments = df_cleaned_news.join(predictions_df, how='inner')

In [20]:
news_sentiments.head()

,Unnamed: 0,date,clean_text,clean_title,"(positive,)","(negative,)"
0,0,2022-01-06,chicago ap chicago school leaders canceled classes thursday second consecutive day failing reach agreement teachers union covid safety protocols nation third largest school district the chicago teachers union sought revert remote instruction latest surge infections sides hammer deal but chicago public schools leaders said remote learning work schools safely remain open protocols place the move cancel classes activities thursday affects roughly students ...,no deal school chicago cancels classes 2nd day,0.295749,0.957052
1,1,2022-01-06,chicago pd lies season episode promo voight employs new informant help solve tricky drug trafficking case atwater struggles reconcile personal professional life reaches decision jesseleesoffer marinasqu trspiridakos nbcchicagopd,watch chicago p d preview wednesday,0.002615,0.986636
2,2,2022-01-06,wise foolish the chicago tribune article titled chicago public schools cancels classes thursday deadlock union continues trump biden press secretary comment impasse reported trumps solution follows educate children person give every dollar spent education directly students get failing government schools trump said statement the purpose this post is relate ancient wisdom proverb trump solution king solomon go presence foolish man thou perceivest lips...,trump solution chicago public schools king solomon blog,0.040025,0.966754
3,3,2022-01-06,the neighbors near mashawn plummer portage park firehouse knew often sat outside friendly presence station he showed early shifts ready job thrilled friends family recalled chicago firefighter emt mashawn plummer engine chicago fire department hundreds chicago firefighters joined family friends thursday lay plummer rest died december battling apartment fire northwest side they remembered plummer gentle giant followed dream becoming chicago fi...,family friends colleagues gathering today south side church attend funeral services chicago firefighter mashawn plummer,0.892183,0.033660
4,4,2022-01-06,mashawn plummer died final month preventing fireplace town northwest aspect additionally killed civilian a visitation held plummer wednesday night time a remaining visitation probably held home hope church e 114th st related chicago firefighter amongst critically injured belmont central residence fireplace killed following ultimate visitation funeral companies start probably adopted procession oak woods cemetery 67th road the yea...,chicago firefighter funeral pals household say goodbye mashawn plummer,0.915046,0.068743


In [21]:
news_sentiments = news_sentiments.rename(columns={news_sentiments.columns[4]: "positive", \
                                                  news_sentiments.columns[5]: "negative"})


In [22]:
news_sentiments.to_csv("zero_shot_sentiments.csv")

In [23]:
news_sentiments.shape

(165360, 6)

In [24]:
news_sentiments.head()

,Unnamed: 0,date,clean_text,clean_title,positive,negative
0,0,2022-01-06,chicago ap chicago school leaders canceled classes thursday second consecutive day failing reach agreement teachers union covid safety protocols nation third largest school district the chicago teachers union sought revert remote instruction latest surge infections sides hammer deal but chicago public schools leaders said remote learning work schools safely remain open protocols place the move cancel classes activities thursday affects roughly students ...,no deal school chicago cancels classes 2nd day,0.295749,0.957052
1,1,2022-01-06,chicago pd lies season episode promo voight employs new informant help solve tricky drug trafficking case atwater struggles reconcile personal professional life reaches decision jesseleesoffer marinasqu trspiridakos nbcchicagopd,watch chicago p d preview wednesday,0.002615,0.986636
2,2,2022-01-06,wise foolish the chicago tribune article titled chicago public schools cancels classes thursday deadlock union continues trump biden press secretary comment impasse reported trumps solution follows educate children person give every dollar spent education directly students get failing government schools trump said statement the purpose this post is relate ancient wisdom proverb trump solution king solomon go presence foolish man thou perceivest lips...,trump solution chicago public schools king solomon blog,0.040025,0.966754
3,3,2022-01-06,the neighbors near mashawn plummer portage park firehouse knew often sat outside friendly presence station he showed early shifts ready job thrilled friends family recalled chicago firefighter emt mashawn plummer engine chicago fire department hundreds chicago firefighters joined family friends thursday lay plummer rest died december battling apartment fire northwest side they remembered plummer gentle giant followed dream becoming chicago fi...,family friends colleagues gathering today south side church attend funeral services chicago firefighter mashawn plummer,0.892183,0.033660
4,4,2022-01-06,mashawn plummer died final month preventing fireplace town northwest aspect additionally killed civilian a visitation held plummer wednesday night time a remaining visitation probably held home hope church e 114th st related chicago firefighter amongst critically injured belmont central residence fireplace killed following ultimate visitation funeral companies start probably adopted procession oak woods cemetery 67th road the yea...,chicago firefighter funeral pals household say goodbye mashawn plummer,0.915046,0.068743
